<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
#!wget httpdrive.mount('/content/gdrive')s://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
#!unzip gdrive/MyDrive/concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained modeJust like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [3]:
!pip install keras.applications 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# Importing neccesary libraries
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import load_model
import numpy as np

In [5]:
# Setting up variables to use as parameters
num_classes = 2
image_resize = 224
batch_size_training = 100
batch_size_validation = 100

In [6]:
# Creating an ImageDataGenerator for the training set
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [7]:
# Training generator
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [8]:
# Validation generator
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size = (image_resize, image_resize),
    batch_size = batch_size_validation,
    class_mode = 'categorical'
)

Found 9501 images belonging to 2 classes.


In [9]:
# Creating a sequential model using Keras with a VGG16 model and a dense layer
vgg16_model = Sequential()
vgg16_model.add(VGG16(
    include_top = False,
    pooling = 'avg',
    weights = 'imagenet'
))
vgg16_model.add(Dense(num_classes, activation = 'softmax'))
vgg16_model.layers[0].trainable = False

In [10]:
vgg16_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                                 
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [11]:
# Compiling the model using adam as optimizer and cross entropy as the loss function
vgg16_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [12]:
# Defining the epochs to train the model
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [13]:
# Begin training the model
fit_history = vgg16_model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

<ipython-input-13-6e2f54e79882>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fit_history = vgg16_model.fit_generator(


Epoch 1/2
301/301 [==============================] - 186s 563ms/step - loss: 0.0965 - accuracy: 0.9682 - val_loss: 0.0241 - val_accuracy: 0.9962
Epoch 2/2
301/301 [==============================] - 172s 572ms/step - loss: 0.0190 - accuracy: 0.9961 - val_loss: 0.0144 - val_accuracy: 0.9974


In [14]:
# Save our model
vgg16_model.save('classifier_vgg16_model.h5')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [15]:
# Previously created model
resnet_model = load_model('/content/gdrive/MyDrive/classifier_resnet_model.h5')

In [16]:
# Construction an ImageDataGenerator for the testing set
test_data_generator = ImageDataGenerator()

test_train_generator = test_data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    shuffle = False)

Found 30001 images belonging to 2 classes.


In [17]:
# Evaluating the VGG16 model
eval_vgg16 = vgg16_model.evaluate_generator(test_train_generator)
print('Performance for model VGG16:')
print('Loss: ', (eval_vgg16[0]))
print('Accuracy: ', (eval_vgg16[1]))

<ipython-input-17-1120c4bef708>:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  eval_vgg16 = vgg16_model.evaluate_generator(test_train_generator)


Performance for model VGG16:
Loss:  0.18522822856903076
Accuracy:  0.9262024760246277


In [18]:
# Evaluating the ResNet model
eval_resnet = resnet_model.evaluate_generator(test_train_generator)
print('Performance for model ResNet50:')
print('Loss: ', (eval_resnet[0]))
print('Accuracy: ', (eval_resnet[1]))

<ipython-input-18-0e410b403c4f>:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  eval_resnet = resnet_model.evaluate_generator(test_train_generator)


Performance for model ResNet50:
Loss:  0.10654720664024353
Accuracy:  0.9563014507293701


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [19]:
y_hat_vgg16 = vgg16_model.predict_generator(test_train_generator)
y_hat_resnet = resnet_model.predict_generator(test_train_generator)

<ipython-input-19-4d6b127ca8e2>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  y_hat_vgg16 = vgg16_model.predict_generator(test_train_generator)
<ipython-input-19-4d6b127ca8e2>:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  y_hat_resnet = resnet_model.predict_generator(test_train_generator)


In [20]:
# Helper function to predict the class of an image
def predict(batch):
  for iterator in batch:
     prediction = np.argmax(iterator)
     if(prediction == 0):
       print('Negative')
     else:
        print('Positive')

In [21]:
print('Report the class predictions of the first five images in the test set for VGG16:')
predict(y_hat_vgg16[0:5])

Report the class predictions of the first five images in the test set for VGG16:
Negative
Negative
Negative
Negative
Negative


In [22]:
print('Report the class predictions of the first five images in the test set for ResNet50:')
predict(y_hat_resnet[0:5])

Report the class predictions of the first five images in the test set for ResNet50:
Negative
Negative
Negative
Negative
Negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).